In [ ]:
!pip install pymupdf reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.3 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
raw_path = "/content/raw_data"

Проверка и очистка данных

In [ ]:
import os
from PyPDF2 import PdfReader

def check_pdf_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            try:
                path = os.path.join(directory, filename)
                PdfReader(path)  # Пробуем открыть файл
                print(f"{filename}: OK")
            except Exception as e:
                print(f"{filename}: Error - {e}")

check_pdf_files(raw_path)


2C45NWO7FMTDRTBBR6FA442OZKSCLGZZ.pdf: OK
25OOF5WZSJ4M6JG3BPYDO56GYBR65WET.pdf: OK
2426HG2RXRZNRHX3YHORDF5C57PPAAHE (1).pdf: OK
2426HG2RXRZNRHX3YHORDF5C57PPAAHE (1) (1).pdf: OK
2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf: OK
27CRLC3DDVOHBJ27UZSHYXRO3OB4BHIY.pdf: OK
2NYYOQN3UKYWMKXDAMTWP4ZNMGBBTLCA.pdf: OK
2IOECB7XKI56SDSRDXHXTAUDGCMYTVI7.pdf: OK
business-policy.pdf: OK


Удаление пропусков и пустых слайдов

In [ ]:
import fitz  # PyMuPDF

def check_empty_slides(directory):
    empty_slides = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            path = os.path.join(directory, filename)
            doc = fitz.open(path)

            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()
                if not text.strip():  # Если текст пустой
                    empty_slides.append((filename, page_num + 1))

    if empty_slides:
        print("Пустые слайды:")
        for slide in empty_slides:
            print(f"Файл: {slide[0]}, Слайд: {slide[1]}")
    else:
        print("Пустых слайдов не найдено.")

check_empty_slides(raw_path)


Пустые слайды:
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 21
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 22
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 23
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 24
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 25


Выбросы

In [ ]:
def check_image_only_pages(directory):
    image_only_pages = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            path = os.path.join(directory, filename)
            doc = fitz.open(path)

            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()

                # Проверим, есть ли текст или только изображения
                if not text.strip() and len(page.get_images(full=True)) > 0:
                    image_only_pages.append((filename, page_num + 1))

    if image_only_pages:
        print("Страницы с изображениями, без текста:")
        for page in image_only_pages:
            print(f"Файл: {page[0]}, Слайд: {page[1]}")
    else:
        print("Нет страниц с изображениями без текста.")

check_image_only_pages(raw_path)


Страницы с изображениями, без текста:
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 21
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 22
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 23
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 24
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 25


Отчет об ошибках

In [ ]:
import json

def generate_clean_report(directory):
    report = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            path = os.path.join(directory, filename)
            doc = fitz.open(path)
            file_report = {"file": filename, "empty_pages": []}

            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()
                if not text.strip():
                    file_report["empty_pages"].append(page_num + 1)

            if file_report["empty_pages"]:
                report.append(file_report)
    print(report)
    # Сохранить отчет в JSON
    with open("cleaning_report.json", "w") as f:
        json.dump(report, f, indent=4)

generate_clean_report(raw_path)


[{'file': '2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf', 'empty_pages': [21, 22, 23, 24, 25]}]


In [ ]:
import os
import re
from datetime import datetime

def rename_presentation_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            # Получаем полный путь
            old_path = os.path.join(directory, filename)

            # Убираем расширение файла
            base_name = os.path.splitext(filename)[0]

            # Применяем преобразования для улучшения читаемости имени
            # Пример: удаление ненужных символов, замена пробелов на нижнее подчеркивание и приведение к нижнему регистру
            new_base_name = re.sub(r'[^\w\s]', '', base_name)  # Убираем все спецсимволы
            new_base_name = new_base_name.replace(" ", "_")  # Заменяем пробелы на подчеркивания
            new_base_name = new_base_name.lower()  # Приводим к нижнему регистру

            # Добавление метки времени (например, если у вас есть несколько версий одной презентации)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            new_name = f"{new_base_name}_{timestamp}.pdf"

            # Новый путь для файла
            new_path = os.path.join(directory, new_name)

            # Переименование файла
            os.rename(old_path, new_path)
            print(f"Файл '{filename}' переименован в '{new_name}'")

# Пример использования
rename_presentation_files(raw_path)


Файл '2C45NWO7FMTDRTBBR6FA442OZKSCLGZZ.pdf' переименован в '2c45nwo7fmtdrtbbr6fa442ozksclgzz_20241209_155313.pdf'
Файл '25OOF5WZSJ4M6JG3BPYDO56GYBR65WET.pdf' переименован в '25oof5wzsj4m6jg3bpydo56gybr65wet_20241209_155313.pdf'
Файл '2426HG2RXRZNRHX3YHORDF5C57PPAAHE (1).pdf' переименован в '2426hg2rxrznrhx3yhordf5c57ppaahe_1_20241209_155313.pdf'
Файл '2426HG2RXRZNRHX3YHORDF5C57PPAAHE (1) (1).pdf' переименован в '2426hg2rxrznrhx3yhordf5c57ppaahe_1_1_20241209_155313.pdf'
Файл '2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf' переименован в '2l5jfjnvcbuld4v5ufnmpukwm3byssyc_1_20241209_155313.pdf'
Файл '27CRLC3DDVOHBJ27UZSHYXRO3OB4BHIY.pdf' переименован в '27crlc3ddvohbj27uzshyxro3ob4bhiy_20241209_155313.pdf'
Файл '2NYYOQN3UKYWMKXDAMTWP4ZNMGBBTLCA.pdf' переименован в '2nyyoqn3ukywmkxdamtwp4znmgbbtlca_20241209_155313.pdf'
Файл '2IOECB7XKI56SDSRDXHXTAUDGCMYTVI7.pdf' переименован в '2ioecb7xki56sdsrdxhxtaudgcmytvi7_20241209_155313.pdf'
Файл 'business-policy.pdf' переименован в 'businesspolicy_202412